In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


<img src="./MNIST.png" align="left" width="300"/>

MNIST 손글씨 실습을 위해서 케라스에서 제공하는 MNIST 데이터 셋을 사용한다.

학습 데이터에는 60,000개의 샘플 데이터가 있고 테스트 데이터에는 총 10,000개의 샘플 데이터가 있다.  
MNIST 손글씨 데이터는 이미지 하나가 28개의 행과 28개의 열을 가지는 픽셀 데이터이고 각 픽셀은 흑백 사진과 같이 0부터 255까지의 그레이스케일을 가지고 있다.

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print('x_train.shape: {}, y_train.shape: {}'.format(x_train.shape, y_train.shape))
print('x_test.shape: {}, y_test.shape: {}'.format(x_test.shape, y_test.shape))

x_train.shape: (60000, 28, 28), y_train.shape: (60000,)
x_test.shape: (10000, 28, 28), y_test.shape: (10000,)


학습 데이터를 학습 데이터(5만개)와 검증 데이터(1만개)로 분리한다.  
학습 중간마다 검증 데이터로 모델 성능을 측정하면 모델 학습이 제대로 진행되는지 검증 정확도를 알 수 있고 학습 정확도는 올라가는데 검증 정확도가 더 이상 올라가지 않거나 오히려 떨어질 경우 학습의 조기 종료를 구현할 수 있다.

In [3]:
# 검증 데이터로 사용하기 위해서 학습 데이터에서 1만개를 분리한다.
x_val = x_train[50000:] # 28행 28열로 구성된 검증 데이터
x_train = x_train[:50000] # 28행 28열로 구성된 학습 데이터
y_val = y_train[50000:] # 검증 데이터 레이블
y_train = y_train[:50000] # 학습 데이터 레이블
print('x_train.shape: {}, y_train.shape: {}'.format(x_train.shape, y_train.shape)) # 학습
print('x_val.shape: {}, y_val.shape: {}'.format(x_val.shape, y_val.shape)) # 학습 중 검증
print('x_test.shape: {}, y_test.shape: {}'.format(x_test.shape, y_test.shape)) # 학습 후 테스트

x_train.shape: (50000, 28, 28), y_train.shape: (50000,)
x_val.shape: (10000, 28, 28), y_val.shape: (10000,)
x_test.shape: (10000, 28, 28), y_test.shape: (10000,)


학습 데이터를 출력해보면 데이터가 0부터 255사이의 숫자(그레이스케일)로 구성된 것을 확인할 수 있다.

In [4]:
print(y_train[:10])
for i in x_train[0]:
    for j in i:
        print('{:3d} '.format(j), end='')
    print()

[5 0 4 1 9 2 1 3 1 4]
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136 175  26 166 255 247 127   0   0   0   0 
  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253 225 172 253 242 195  64   0   0   0   0 
  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251  93  82  82  56  39   0   0   0   0   0 
  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241   

다층 퍼셉트론을 사용한 손글씨 이미지 분류 작업 흐름도

<img src="./MNIST2.png" align="left" width="1300"/>

다층 퍼셉트론의 입력값은 무조건 1차원 형태의 배열만 가능하다.  
MNIST 손글시 데이터를 다층 퍼셉트론의 입력값으로 사용할 수 있도록 넘파이의 reshape() 메소드를 사용해서 2차원 배열 형태의 데이터를 1차원 배열 형태로 변경한다.

In [5]:
print('x_train.shape: {}'.format(x_train.shape))
# 28행 28열로 구성된 학습 데이터를 784개의 1차원 배열 형태로 변환한다.
# x_train = np.reshape(x_train, [50000, 784])
x_train = x_train.reshape(50000, 784)
# x_train = x_train.reshape(-1, 784)
print('x_train.shape: {}'.format(x_train.shape))
# 28행 28열로 구성된 검증 데이터를 784개의 1차원 배열 형태로 변환한다.
x_val = x_val.reshape(10000, 784)
print('x_val.shape: {}'.format(x_val.shape))
# 28행 28열로 구성된 테스트 데이터를 784개의 1차원 배열 형태로 변환한다.
x_test = x_test.reshape(10000, 784)
print('x_test.shape: {}'.format(x_test.shape))

x_train.shape: (50000, 28, 28)
x_train.shape: (50000, 784)
x_val.shape: (10000, 784)
x_test.shape: (10000, 784)


1차원으로 변경된 데이터를 그대로 다층 퍼셉트론의 입력되도 되지만 조금 더 효율적인 학습을 위해서 데이터를 정규화 한다.  
데이터를 정규화 하면 모델의 학습 시간을 단축시키고 더 나은 성능을 보이는 효과가 있다.  
MNIST 손글씨 데이터의 모든 값들은 0부터 255사이의 범위 안에 있으므로 255로 나눠 모든 값들이 0부터 1사이의 값으로 정규화 된다.

In [6]:
# MNIST 손글씨 데이터의 각 픽셀의 데이터 타입은 부호 없이 0부터 255사이의 값만 기억하면 되므로 데이터 타입이
# 부호없는 8비트 정수(uint8)로 되어있다.
# print(type(x_train[0][0])) # <class 'numpy.uint8'>
# 255로 나눠서 0부터 1사이의 실수로 만들어야 하므로 astype() 메소드로 실수로 변환한다.
x_train = x_train.astype(np.float32)
# print(type(x_train[0][0])) # <class 'numpy.float32'>
x_train /= 255
x_val = x_val.astype(np.float32)
x_val /= 255
x_test = x_test.astype(np.float32)
x_test /= 255